Donades les [dades de la conselleria de Salut](https://analisi.transparenciacatalunya.cat/Salut/Vacunaci-per-al-COVID-19-dosis-administrades-per-r/tp23-dey4) del 1 d'agost del 2021 sobre vacunacions calcula:

- **1.** Quantes persones s'an vacunat amb la primera vacuna? I amb la segona? I quantes persones han demanat cita i no han assistit?
- **2.** Mostra un gràfic que mostri la població vacunada per edats amb primera i segona dosi.
- **3.** Mostra el gràfic anterior mostrant els percentatges de grups utilitzant les [dades de l'1 de gener de 2021 sobre població per edats de l'IDESCAT](https://analisi.transparenciacatalunya.cat/Salut/Vacunaci-per-al-COVID-19-dosis-administrades-per-r/tp23-dey4) .
- **4.** Mostra un gràfic amb el número de vacunes administrades segons cada fabricant.
- **5.** Mostra un gràfic on es superposi la suma dels vacunats amb pauta complta acumulada amb el número de vacunes administrades diàriament.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from datetime import datetime

%matplotlib inline 

### Import CSV from local 

In [2]:
# handlig dates
dateparse = lambda x: datetime.strptime(x, '%d/%m/%Y')

In [ ]:
# Import CSV from local 
vac_csv = f"C:/capes_sql/post_covid/vacunacio.csv"
vac_df = pd.read_csv(vac_csv, sep=',',parse_dates=['DATA'], date_parser=dateparse)
vac_df.head(2)

### **1.** Quantes persones s'han vacunat amb la primera vacuna? I amb la segona? I quantes persones han demanat cita i no han assistit?

In this dataset the people with unfavorable medical indications for vaccination are registered as NO_VACUNAT

In [ ]:
# people non vaccinated
vac_by_prod = vac_df.groupby(["NO_VACUNAT"])[["RECOMPTE"]].sum() 
vac_by_prod.head()

In [ ]:
# drop Non vaccinated people rows by condition
vac_df = vac_df[vac_df['NO_VACUNAT'] != "No vacunat"]

In [ ]:
# sum "RECOMPTE" by first dose "DOSI"
vac_df.query("DOSI == 1")['RECOMPTE'].sum()

In [ ]:
# sum "RECOMPTE" by second dose "DOSI"
vac_df.query("DOSI == 2")['RECOMPTE'].sum()

### **2.** Mostra un gràfic que mostri la població vacunada per edats amb primera i segona dosi.

In [ ]:
# Select rows with first dose
vac_first_dose = vac_df[vac_df.DOSI == 1]
#group by age and sum RECOMPTE
vac_first_dose_by_age = vac_first_dose.groupby(["EDAT"])[["RECOMPTE"]].sum()
# Rename RECOMPTE column to DOSIU
vac_first_dose_by_age.rename(columns = {'RECOMPTE':'DOSI_U'}, inplace = True)

# Select rows with second dose
vac_second_dose = vac_df[vac_df.DOSI == 2]
# Group by age and sum RECOMPTE
vac_second_dose_by_age = vac_second_dose.groupby(["EDAT"])[["RECOMPTE"]].sum()
# Rename RECOMPTE column to DOSIDOS
vac_second_dose_by_age.rename(columns = {'RECOMPTE':'DOSI_DOS'}, inplace = True)

In [ ]:
vac_second_dose_by_age.head()

In [ ]:
# merge the two dataframe
vac_by_age_m = pd.merge(vac_first_dose_by_age, vac_second_dose_by_age, on='EDAT')
vac_by_age_m

In [ ]:
# plot the df
fig, ax = plt.subplots(figsize=(15, 7))

labels = vac_by_age_m.index #labels x-axis 

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

d1 = ax.bar(x - width/2 - 0.04, #bars position
            vac_by_age_m.DOSI_U, 
            width,
            label='First dose', 
            color="#00cc66")

d2 = ax.bar(x + width/2, vac_by_age_m.DOSI_DOS, width, label='Second dose')

ax.set_xticks(x)
ax.set_xticklabels(labels) # custom labels x-axis

ax.bar_label(d1, #d1 = bar container
             labels= vac_by_age_m.DOSI_U//1000, # // int division
             padding=2, fontsize=12)
ax.bar_label(d2, labels= vac_by_age_m.DOSI_DOS//1000, padding=2, fontsize=12)

ax.get_yaxis().set_major_formatter(
matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

plt.title("Number of vaccines administrated grouped by age",  fontsize=20)
plt.xticks(rotation=45, fontsize=14) # rotation axis tags
plt.yticks(fontsize=14)
plt.legend(loc='upper left', fontsize=14)

plt.show()

### 3. Mostra el gràfic anterior mostrant els percentatges de grup utilitzant les [dades de l'1 de gener de 2021 sobre població per edats de l'IDESCAT](https://analisi.transparenciacatalunya.cat/Salut/Vacunaci-per-al-COVID-19-dosis-administrades-per-r/tp23-dey4) .

In [ ]:
pop_by_age = [1157226, 411266, 402872, 434341, 461705, 522779, 643156, 649531, 581232,
              521531, 459344, 389980, 352045, 279754, 449998]
df_pop = pd.DataFrame(pop_by_age)


In [ ]:
vac_by_age_m = vac_by_age_m.drop(['No classificat'], axis=0) # remove values non classified
vac_by_age_m.index.name = None #remove tag axis
vac_by_age_m['POP'] = pop_by_age  # add list as column

In [ ]:
# % first dose
vac_by_age_m['XDOSI_U'] =  vac_by_age_m['DOSI_U'] / vac_by_age_m['POP'] * 100
# % second dose
vac_by_age_m['XDOSI_D'] =  vac_by_age_m['DOSI_DOS'] / vac_by_age_m['POP'] * 100

In [ ]:
vac_by_age_m

In [ ]:
# plot the df
fig, ax = plt.subplots(figsize=(17, 7))

labels = vac_by_age_m.index #labels x-axis 

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

d1 = ax.bar(x - width/2 - 0.04, #bars position
            vac_by_age_m.XDOSI_U, 
            width,
            label='First dose', 
            color="#00cc66")

d2 = ax.bar(x + width/2, vac_by_age_m.XDOSI_D, width, label='Second dose')

ax.set_xticks(x)
ax.set_xticklabels(labels) # custom labels x-axis

# bar labels
vac_round = vac_by_age_m.round(1)
ax.bar_label(d1, labels= vac_round.XDOSI_U, padding=2, fontsize=12)
ax.bar_label(d2, labels= vac_round.XDOSI_D, padding=2, fontsize=12)


plt.title("Percentage of vaccines administrated by group of age",  fontsize=20)
plt.xticks(rotation=45, fontsize=14) # rotation axis tags
plt.yticks(fontsize=14)
plt.legend(loc='upper left', fontsize=17)

plt.show()

# **4.** Mostra un gràfic amb el número de vacunes administrades segons cada fabricant.

In [ ]:
#group by FABRICANT and sum RECOMPTE
vac_by_prod = vac_df.groupby(["FABRICANT"])[["RECOMPTE"]].sum()
vac_by_prod.head()

In [ ]:
explode = (0.01, 0, 0, 0)  # only "explode" the data

# Creating color parameters
colors = ( "orange", "cyan", "brown","magenta")

fig1, ax1 = plt.subplots(figsize=(10,7))

ax1.pie(vac_by_prod.RECOMPTE, 
        explode=explode, 
        labels=vac_by_prod.index,  
        autopct='%1.1f%%' ,
        shadow=True, 
        startangle=120,
       colors=colors,
       textprops = dict(color ="black", fontsize=17, family='sans-serif'))

ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.title("Vacunes administrades segons el fabricant",  fontsize=24,  y=1.08) # y avoid overlaps
plt.xticks(fontsize=14)


plt.tight_layout()
plt.show()

## 5. Mostra un gràfic on es superposi la suma dels vacunats amb pauta complta acumulada amb el número de vacunes administrades diàriament.


In [ ]:
 vac_df.head()

In [ ]:
# selection DOSI 1 
# Select rows with first dose
vac_first_dose = vac_df[vac_df.DOSI == 1]
#group by DATA and sum RECOMPTE
vac_first_dose_by_date = vac_first_dose.groupby(["DATA"])[["RECOMPTE"]].sum()

# selection DOSI 2
# Select rows with second dose
vac_second_dose = vac_df[vac_df.DOSI == 2]
#group by DATA and sum RECOMPTE
vac_second_dose_by_date = vac_second_dose.groupby(["DATA"])[["RECOMPTE"]].sum()

In [ ]:
# Rename each column - appending “insitu”
vac_second_dose_by_date.rename(columns={"RECOMPTE": "REC_DOS"},
                      inplace=True)  # Modify the dataframe columns

vac_second_dose_by_date.tail()

In [ ]:
#remove index axis tag
vac_first_dose_by_date.index.name = None 
vac_second_dose_by_date.index.name = None

In [ ]:
vac_first_dose_by_date.tail()

In [ ]:
# Joining on index 
df_vac_app =vac_first_dose_by_date.join(vac_second_dose_by_date, how="outer")
df_vac_app

In [ ]:
df_vac_app = df_vac_app.fillna(0) # fill the null values as 0

In [ ]:
# cummulative sum second dose
vac_second_dose_by_date["cumsum2"] = vac_second_dose_by_date['REC_DOS'].cumsum()

# cummulative sum second dose
vac_first_dose_by_date["cumsum"] = vac_first_dose_by_date['RECOMPTE'].cumsum()

In [ ]:
df_vac_app

In [ ]:
# Plot your data
fig, ax = plt.subplots(figsize=(13,7))

width_col = 0.6
labels = df_vac_app.index
    
ax.bar(labels, df_vac_app.RECOMPTE, width_col, label='Primera dosi', color='#009933') # first dose
ax.bar(labels, df_vac_app.REC_DOS, width_col, 
       bottom=df_vac_app["RECOMPTE"],
       label='Segona dosi', 
     color='#0000cc')


ax.set_ylabel('Númro de vacunes administrades diàriament')
ax.set_title('Scores by group and gender')
ax.legend()

In [ ]:
# Plot your data
fig, ax = plt.subplots(figsize=(11,7))
vac_second_dose_by_date["cumsum2"].plot(ax=ax, label = "Cumulative Completed Vaccination")

# Make the y-axis label, ticks and tick labels match the line color.
ax.set_ylabel('Número de persones vacunades \n amb dos dosis', fontsize=18,  color='b')



# Turn off scientific notation
#ax.ticklabel_format(style='plain')